# Day 1

In [46]:
with open('day1/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [63]:
arr = [int(x) for x in arr]

def two_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    while l < h:
        if arr[l] + arr[h] == x:
            return arr[l] * arr[h]
        elif arr[l] + arr[h] < x:
            l += 1
        else:
            h -= 1
    return -1


In [65]:
def three_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    for y in arr:
        tmp = two_sum(arr, x - y)
        if tmp != -1:
            return tmp * y

In [67]:
two_sum(arr, 2020)

960075

In [68]:
three_sum(arr, 2020)

212900130

# Day 2

In [22]:
import re

# part 1
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        if len(re.findall(letter, pwd)) in range(l, h + 1):
            n += 1
            
print(f"For Part 1 there are {n} correct passwords")

There are 625 correct passwords


In [43]:
# part 2
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        try:
            l_letter = pwd[l-1]
            h_letter = pwd[h-1]
        except IndexError:
            continue
        
        if (l_letter == letter) != (h_letter == letter):
            n += 1

In [44]:
print(f"For Part 2 there are {n} correct passwords")

For Part 2 there are 391 correct passwords
